In [42]:
import numpy as np
import timeit
from sndaq.datahandler import DataHandler
from sndaq.detector import Detector
from sndaq.analysis import AnalysisHandler
from sndaq.datetime_ns import datetime_ns as dt_ns

In [43]:
dh = DataHandler()
i3 = Detector('../data/config/full_dom_table.txt')
ana = AnalysisHandler()

In [44]:
start_time = timeit.default_timer()
dh.get_data_files('../data')
for file in dh.files:
    print(f'Processing {file}')
    dh.set_file(file)
    ### Process file
    # Setup variables from first payload in file
    dh.read_payload()
    while not i3.isvalid_dom(dh.payload.dom_id):
        dh.read_payload()

    utime = dh.payload.utime
    dh._file_start_utime = utime
    if dh._start_utime is None and utime is not None:
        dh._start_utime = utime
        dh._raw_utime = np.arange(utime, utime + (dh._raw_udt * dh._staging_depth), dh._raw_udt)

    while dh.payload is not None:
        
        # Advance buffer after 2 ms bin is ready
        while dh.payload is not None and dh.payload.utime <= dh._raw_utime[1]:
            idx_dom = i3.get_dom_idx(dh.payload.dom_id)
            dh.update_buffer(idx_dom)  #Ensures first payload is added to buffer
            dh.read_payload()
            
            while dh.payload is not None and not i3.isvalid_dom(dh.payload.dom_id):  # Skip IceTop
                dh.read_payload()

        if dh._pay is not None: 
            # dh._pay is None only if EOF is reached, this ensures raw bins that span files
            # receive all contributing scalers.
            ana.update(dh._data[:,0])
            # Eval trigger
            dh.advance_buffer()
    
    # If payload is none, then EOF reached. Close file
    dh._file.close()
elapsed = timeit.default_timer() - start_time
print('Elapsed time: {0:3.6f}'.format(elapsed))
processed = dt_ns.from_utime(dh._raw_utime[0]) -  dt_ns.from_utime(dh._start_utime)
print('Processed time: {0:3.6f}'.format(processed.to_ns/1e9))
print('Ratio: {0:3.6f}'.format(elapsed/(processed.to_ns/1e9)))

Processing ../data/sn_209058_000001_239420_529095.dat
Processing ../data/sn_209058_000002_529096_818955.dat
Processing ../data/sn_209058_000003_818956_1108882.dat
Elapsed time: 554.4396316089842
Processed time: 175.922


### sni3daq prototype performance
The above cells represent the prototype version of python SNDAQ

Performance
- 09/01/21: Ratio (3.152) : 3 Files (~573 MB), representing 175.92s of data, was processed in 554.43s 